# RAG 

#### Libraries

In [2]:
# processing:
import os
import re
import pandas as pd
from dotenv import load_dotenv

# neo4j:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector

# llm:
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings import GigaChatEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# retrieval:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory

#### .env

In [266]:
load_dotenv()

True

In [267]:
# neo4j:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE= os.getenv('NEO4J_DATABASE')

# llm:
LLM_SCOPE = os.getenv('SCOPE')
LLM_AUTH = os.getenv('AUTH_DATA')

# telegram:
BOT_TOKEN = os.getenv('BOT_TOKEN')

#### Reference

In [268]:
reference = pd.read_excel('../docs/reference.xlsx')

## Setting Up LLM | GigaChat


[GigaChat](https://developers.sber.ru/docs/ru/gigachat/overview)

In [269]:
llm = GigaChat(credentials = LLM_AUTH, 
               temperature = 0.3, 
               n = 1, 
               model = "GigaChat-Plus", # 32k context window
               repetition_penalty = 1.0,
               verify_ssl_certs = False)

In [270]:
parser = StrOutputParser()

In [271]:
embeddings = GigaChatEmbeddings(credentials = LLM_AUTH, verify_ssl_certs = False)

## Prompt Engineering

#### Prompt Template

In [272]:
template = """
            Задача: Анализировать заданный запрос и предоставлять детализированные ответы, опираясь на документы, правила и требования.

            Инструкции:

            1. При ответе на вопросы, особенно те, которые связаны с правилами или нормативными документами, активно ссылайтесь на номера пунктов, статей и разделов документов.
            2. Если требуемая информация отсутствует или вы не можете обнаружить необходимые пункты в документах, используйте фразу "Информация не найдена".
            3. Ваши ответы должны быть максимально точными и содержать не только ссылки на документы, но и объяснения их применения к данному запросу.

            Вопрос: {question}
            Контекст: {context}
            
            Ответ: Предоставьте ваш ответ, опираясь на указанные выше указания.
            """

prompt = ChatPromptTemplate.from_template(template)

#### Step Back Prompting

In [273]:
def step_back_prompt(model, parser, text):
    
    chain = model | parser

    template = f"""
                Задача: Анализировать представленные данные и вывести из них высокоуровневые концепции и основные принципы.

                Инструкции:
                Прежде чем приступить к абстракции, важно:
                1. Определить и выделить ключевые детали и специфику представленного материала.
                2. Анализировать связи между деталями для понимания более глубоких закономерностей и взаимосвязей.
                3. Структурировать полученные данные, выделяя общие элементы и паттерны.
                4. Формулировать высокоуровневые концепции и принципы, опираясь на проанализированные данные.

                Текст: \"{text}\"

                Проанализируйте и сформулируйте общие концепции и основные принципы на основе представленных деталей в виде развернутых вопросов.
                """

    model_response = chain.invoke(template)
    
    return model_response

#### Question Extraction

In [274]:
def extract_question(model, parser, text):
    
    chain = model | parser

    template = f"""
                            Мне нужно твоё содействие в анализе следующего делового письма.
                            Извлеки из него всю важную информацию для системы RAG. 
                            Нужны вопросы, ключевые параметры, и основные темы обсуждения. Вот текст письма:

                            Текст: \"{text}\"

                            Разбери письмо на следующие компоненты:

                            1. Все поставленные вопросы.
                            2. Перечень ключевых параметров, упомянутых в тексте.
                            3. Основные темы, которые обсуждаются.

                            Используй структурированный подход, чтобы я мог легко использовать эти данные для запросов в системе RAG.
                            Без дополнительных комментариев.
                            """

    model_response = chain.invoke(template)
    
    return model_response

## Setting Up Database | Neo4j

[Neo4j](https://workspace-preview.neo4j.io)

In [275]:
neo4j_vector = Neo4jVector.from_existing_index(
    embeddings,
    url = NEO4J_URI,
    username = NEO4J_USERNAME,
    password = NEO4J_PASSWORD,
    index_name = "vector"
    # search_type = 'hybrid'
)

## LLM Retrival Chain

[GigaChain](https://github.com/ai-forever/gigachain)

In [276]:
chain = RetrievalQAWithSourcesChain.from_chain_type(llm,
                                                    chain_type = "stuff", # "stuff", "map_rerank", "refine"
                                                    retriever = neo4j_vector.as_retriever(),
                                                    return_source_documents = False,
                                                    reduce_k_below_max_tokens = True,
                                                    chain_type_kwargs = {
                                                                        "verbose": False,
                                                                        "prompt": prompt, # step_back_prompt
                                                                        "document_variable_name": "context",
                                                                        "memory": ConversationBufferMemory(
                                                                            memory_key='history',
                                                                            input_key='question'),
                                                                        }
                                                    )

## Test

In [277]:
letter = 5 # from the reference .xlsx num + 2

inquiry = str(reference['letter'][letter].replace('\n', ' '))
refer = str(reference['answer '][letter]).replace('\n', ' ')

abstarction = step_back_prompt(llm, parser, inquiry)
question = extract_question(llm, parser, inquiry)


print(f'Inquiry: {inquiry}\n')
print(f'Reference: {refer}\n')
print(f'Abstraction: {abstarction}\n')

print(f'Question: {question}\n')

chain.invoke(
            {"question": question},
            return_only_outputs = True,
            )

Inquiry: Добрый день! При определении НМЦ закупки услуг по поставке и монтажу здания (не капитальное строительство) какой метод определения НМЦ должен быть? 

Reference: В пункте 4.9.1. Положения о закупках предусмотрено, что "Проектно-сметный метод применяется для определения начальной (максимальной) цены договора (предмета закупки) на строительство, реконструкцию и капитальный ремонт объектов, а также может быть применен при определении начальной (максимальной) цены договора (предмета закупки) на текущий ремонт зданий, сооружений, строений, помещений".

Abstraction: На основе представленных данных можно сформулировать следующие общие концепции и основные принципы:

1. При определении НМЦ закупки услуг по поставке и монтажу здания (не капитальное строительство) необходимо использовать метод определения НМЦ, который соответствует требованиям законодательства о государственных закупках.

2. Для определения НМЦ следует учитывать специфику и особенности объекта закупки, такие как тип здан

{'answer': 'Для определения начальной (максимальной) цены закупки услуг по поставке и монтажу здания (не капитальное строительство) можно использовать метод сопоставимых рыночных цен (анализ рынка) или метод удельных показателей (параметрический). \n\nМетод сопоставимых рыночных цен (анализ рынка) предполагает использование не менее трех цен товара (работы, услуги), предлагаемых различными поставщиками (подрядчиками, исполнителями). Начальная (максимальная) цена договора (предмета закупки) определяется по формуле: НМЦ=𝑣𝑛*𝑖=1𝑛∑Ц𝑖, где: НМЦ - начальная (максимальная) цена договора (предмета закупки), определяемая методом сопоставимых рыночных цен (анализ рынка);𝑣 - количество (объем) закупаемого товара (работы, услуги); 𝑛 - количество значений, используемых в расчете.\n\nМетод удельных показателей (параметрический) предполагает использование действующей цены на ранее закупаемое изделие и показателя основного параметра ранее закупаемого изделия. Начальная (максимальная) цена договора (пре

In [278]:
docs_with_score = neo4j_vector.similarity_search_with_score(question, k = 5)

for doc, score in docs_with_score:
    print("-" * 200)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 200)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Score:  0.944155752658844
истерства экономического  развития Российской Федерации (для продукции, которая может быть отнесена к потребительским товарам, в качествеИЦП𝑖𝑖−1отр применяется индекс потребительских цен);  С - коэффициент сдерживания темпов роста цен, определяемый в соответствии с локальными нормативными актами ПАО «Компания 1» (если применимо). 
4.5.12 В целях определения начальной (максимальной) цены договора (предмета  закупки) методом сопоставимых рыночных цен (анализ рынка) рекомендуется  использовать не менее трех цен товара (работы, услуги), предлагаемых различными  поставщиками (подрядчиками, исполнителями). 
4.5.13 Начальная (максимальная) цена договора (предмета закупки) методом  сопоставимых рыночных цен (анализ рынка) определяется по формуле: НМЦ=𝑣𝑛*𝑖=1𝑛∑Ц𝑖 где: НМЦ 

## Basic Queries | Neo4j

[Cypher](https://neo4j.com/docs/cypher-cheat-sheet/5/auradb-enterprise)

In [279]:
graph = Neo4jGraph(url = NEO4J_URI, username = NEO4J_USERNAME, password = NEO4J_PASSWORD, database = NEO4J_DATABASE)

In [280]:
print(graph.schema)

Node properties are the following:
Chunk {embedding: LIST, id: STRING, text: STRING, source: STRING, title: STRING}
Relationship properties are the following:

The relationships are the following:



In [281]:
cypher = """
  SHOW VECTOR INDEXES
  """
graph.query(cypher)

[{'id': 2,
  'name': 'vector',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Chunk'],
  'properties': ['embedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 4, 15, 8, 9, 28, 786000000, tzinfo=<UTC>),
  'readCount': 168}]

In [282]:
cypher = """
  MATCH (n)
  RETURN count(n)
  """
graph.query(cypher)

[{'count(n)': 1087}]

In [283]:
cypher = """
    MATCH (n:Chunk {title: "1.2"})
    RETURN n.text AS text
    """
graph.query(cypher)

[{'text': '1.2. Термины и определения'}]

In [284]:
highest_scored_content = str(neo4j_vector.similarity_search(inquiry, k=3))

match = re.search(r'\d+(\.\d+)+', highest_scored_content)

if match:
    highest_scored_document_number = match.group(0)
    print("Highest scored document number:", highest_scored_document_number)

else:
    print("No matching number found")

if match:
    cypher_query = f"""
                    MATCH (n)
                    WHERE n.title STARTS WITH '{highest_scored_document_number}'
                    RETURN n.text
                    """

Highest scored document number: 21.2


In [285]:
graph.query(cypher_query)

[{'n.text': '21.2. Закупки проводятся способом - маркетинговые исследования, если иной способ закупки из числа предусмотренных пунктами 6.2.1, 6.2.2, не определен Центральным органом управления закупками на стадии планирования Закупок.'}]